In [2]:
with open("../input.txt") as f:
    input = [line.strip() for line in f.readlines()]
# print(input)

with open("../sample-input.txt") as f:
    sample_input = [line.strip() for line in f.readlines()]
# print(f"Input: {sample_input}")

In [17]:
from random import seed
from intervaltree import Interval, IntervalTree
from numpy import delete, full

rules = input

trees = {}
# Create an IntervalTree
step_tree : IntervalTree = None
curr_map = None
for rule in rules:
    if rule.startswith("seeds:"):
        curr_map = 'bag-to-seed'
        step_tree = IntervalTree()
        seed_ranges = [int(x) for x in rule.split(": ")[1].split(" ")]
        for i in range(0, len(seed_ranges), 2):
            step_tree[seed_ranges[i]:(seed_ranges[i] + seed_ranges[i+1])] = 0
        continue
    if rule.endswith("map:"):
        curr_map = rule.split()[0]
        step_tree = IntervalTree()
        continue
    values = rule.split()
    if len(values) == 0:
        if step_tree is not None:
            trees[curr_map] = step_tree
            step_tree = None
        continue
    dest_start, source_start, length = map(int, values)
    step_tree[source_start:source_start + length] = dest_start - source_start
if step_tree is not None and len(step_tree) > 0:
    trees[curr_map] = step_tree

In [18]:
current_intervals = trees['bag-to-seed']
for step in ['seed-to-soil', 'soil-to-fertilizer', 'fertilizer-to-water', 'water-to-light', 'light-to-temperature', 'temperature-to-humidity', 'humidity-to-location']:
    # print("Step: ", step)
    next_intervals = IntervalTree()
    
    for interval in current_intervals:
        passthru = IntervalTree([interval])
        # print(f"  Interval: {interval}")
        for assoc_interval in trees[step][interval.begin:interval.end]:
            intersection = (max(interval.begin, assoc_interval.begin), min(interval.end, assoc_interval.end))
            passthru.chop(intersection[0], intersection[1])
            next_interval = Interval(intersection[0] + assoc_interval.data, intersection[1] + assoc_interval.data, intersection)
            # print(f"    Assoc interval: {assoc_interval}, next interval: {next_interval}")
            next_intervals.add(next_interval)
        
        # print(f"  Passthru: {passthru}")
        next_intervals.update(passthru)

    current_intervals = next_intervals
    # print(f"  Next intervals: {next_intervals}")
    # print("")

# print(f"Final intervals: {current_intervals}")
print(f"First location: {current_intervals.range().begin}")

First location: 37806486
